***
# chapter 4.  JOIN 
***

## 4-1  EQUI JOIN (INNER JOIN, SIMPLE JOIN)
키값이 일치되는 데이터만 출력 (NULL은 출력되지 않는다)

In [1]:
import cx_Oracle
import pandas as pd
xedb = cx_Oracle.connect('hr/hr@localhost/xe')
cur = xedb.cursor()

### < employees테이블, departments테이블 연결하기 >

In [3]:
df = pd.read_sql("""
        SELECT e.employee_id, e.last_name, d.department_id, d.department_name
        FROM employees e, departments d
        WHERE e.department_id = d.department_id
        """,xedb)
print(df)

     EMPLOYEE_ID    LAST_NAME  DEPARTMENT_ID  DEPARTMENT_NAME
0            200       Whalen             10   Administration
1            201    Hartstein             20        Marketing
2            202          Fay             20        Marketing
3            114     Raphaely             30       Purchasing
4            115         Khoo             30       Purchasing
5            116        Baida             30       Purchasing
6            117       Tobias             30       Purchasing
7            118       Himuro             30       Purchasing
8            119   Colmenares             30       Purchasing
9            203       Mavris             40  Human Resources
10           120        Weiss             50         Shipping
11           121        Fripp             50         Shipping
12           122     Kaufling             50         Shipping
13           123      Vollman             50         Shipping
14           124      Mourgos             50         Shipping
15      

### < 연결된 두 개의 테이블(employees, departments)에 테이블(locations) 추가하기 >

In [4]:
df = pd.read_sql("""
        SELECT e.employee_id, e.last_name, d.department_id, d.department_name, l.city
        FROM employees e, departments d, locations l
        WHERE e.department_id = d.department_id
        AND d.location_id = l.location_id
        """,xedb)
print(df)

     EMPLOYEE_ID    LAST_NAME  DEPARTMENT_ID   DEPARTMENT_NAME  \
0            100         King             90         Executive   
1            101      Kochhar             90         Executive   
2            102      De Haan             90         Executive   
3            103       Hunold             60                IT   
4            104        Ernst             60                IT   
5            105       Austin             60                IT   
6            106    Pataballa             60                IT   
7            107      Lorentz             60                IT   
8            108    Greenberg            100           Finance   
9            109       Faviet            100           Finance   
10           110         Chen            100           Finance   
11           111      Sciarra            100           Finance   
12           112        Urman            100           Finance   
13           113         Popp            100           Finance   
14        

조인에 있는 두 테이블의 열을 결합하는 where절(e.department_id = d.department_id )보다 
하나의 테이블만 참조하는 where절(e.employee_id=100)을 먼저 수행하는 것이 효율적이다.
(ROWID SCAN 유도)

In [8]:
df = pd.read_sql("""
        SELECT e.employee_id, e.last_name, d.department_name, l.city
        FROM employees e, departments d, locations l
        WHERE e.department_id = d.department_id 
        AND d.location_id = l.location_id   
        AND e.employee_id=100  
        """,xedb)
print(df)

   EMPLOYEE_ID LAST_NAME DEPARTMENT_NAME     CITY
0          100      King       Executive  Seattle


## 4-2. NON EQUI JOIN
* 테이블간 키 값이 정확하게 일치하지 않고,범주간 접근이 필요할 때 사용

In [10]:
# 새로운 테이블 생성 및 조회

cur.execute("CREATE TABLE job_grades(grade_level varchar2(3), lowest_sal number, highest_sal number)")
cur.execute("INSERT INTO job_grades VALUES ('A',1000,2999)")
cur.execute("INSERT INTO job_grades VALUES ('B',3000,5999)")
cur.execute("INSERT INTO job_grades VALUES ('C',6000,9999)")
cur.execute("INSERT INTO job_grades VALUES ('D',10000,14999)")
cur.execute("INSERT INTO job_grades VALUES ('E',15000,24999)")
cur.execute("INSERT INTO job_grades VALUES ('F',25000,40000)")

df = pd.read_sql("""
        SELECT * FROM job_grades
        """,xedb)
print(df)

  GRADE_LEVEL  LOWEST_SAL  HIGHEST_SAL
0           A        1000         2999
1           B        3000         5999
2           C        6000         9999
3           D       10000        14999
4           E       15000        24999
5           F       25000        40000


In [11]:
df = pd.read_sql("""
        SELECT e.last_name, e.job_id, e.salary, j.grade_level
        FROM employees e , job_grades j
        WHERE e.salary 
        BETWEEN j.lowest_sal 
        AND j.highest_sal
        """,xedb)
print(df)

       LAST_NAME      JOB_ID   SALARY GRADE_LEVEL
0          Olson    ST_CLERK   2100.0           A
1         Markle    ST_CLERK   2200.0           A
2     Philtanker    ST_CLERK   2200.0           A
3         Landry    ST_CLERK   2400.0           A
4            Gee    ST_CLERK   2400.0           A
5     Colmenares    PU_CLERK   2500.0           A
6         Marlow    ST_CLERK   2500.0           A
7          Patel    ST_CLERK   2500.0           A
8         Vargas    ST_CLERK   2500.0           A
9       Sullivan    SH_CLERK   2500.0           A
10       Perkins    SH_CLERK   2500.0           A
11        Himuro    PU_CLERK   2600.0           A
12         Matos    ST_CLERK   2600.0           A
13      OConnell    SH_CLERK   2600.0           A
14         Grant    SH_CLERK   2600.0           A
15   Mikkilineni    ST_CLERK   2700.0           A
16           Seo    ST_CLERK   2700.0           A
17        Tobias    PU_CLERK   2800.0           A
18      Atkinson    ST_CLERK   2800.0           A


## 4-3 OUTER JOIN
* 키값이 일치된 데이터 (EQUI JOIN)외에도 누락된 데이터(NULL)도 출력
* 예 : employees테이블에 있는 null값을 뽑아내야 한다면 반대쪽 테이블인 departments테이블에 (+)를 넣는다

### < LEFT OUTER JOIN >
#### 전체 사원의 부서명을 파악하고, 부서배치를 받지 않은 사원까지 조회

In [12]:
df = pd.read_sql("""
        SELECT e.last_name, d.department_name
        FROM employees e, departments d
        WHERE e.department_id = d.department_id(+)
        """,xedb)
print(df)

      LAST_NAME  DEPARTMENT_NAME
0        Whalen   Administration
1           Fay        Marketing
2     Hartstein        Marketing
3    Colmenares       Purchasing
4        Himuro       Purchasing
5        Tobias       Purchasing
6         Baida       Purchasing
7          Khoo       Purchasing
8      Raphaely       Purchasing
9        Mavris  Human Resources
10        Grant         Shipping
11     OConnell         Shipping
12       Feeney         Shipping
13        Walsh         Shipping
14        Jones         Shipping
15       McCain         Shipping
16      Everett         Shipping
17         Bell         Shipping
18      Perkins         Shipping
19        Gates         Shipping
20        Dilly         Shipping
21        Chung         Shipping
22       Cabrio         Shipping
23    Dellinger         Shipping
24         Bull         Shipping
25     Sarchand         Shipping
26        Geoni         Shipping
27     Sullivan         Shipping
28       Fleaur         Shipping
29       T

In [13]:
df = pd.read_sql("""
        SELECT e.last_name, e.department_id, d.department_name
        FROM employees e 
        LEFT OUTER JOIN departments d
        ON (e.department_id = d.department_id)
        """,xedb)
print(df)

      LAST_NAME  DEPARTMENT_ID  DEPARTMENT_NAME
0        Whalen           10.0   Administration
1           Fay           20.0        Marketing
2     Hartstein           20.0        Marketing
3    Colmenares           30.0       Purchasing
4        Himuro           30.0       Purchasing
5        Tobias           30.0       Purchasing
6         Baida           30.0       Purchasing
7          Khoo           30.0       Purchasing
8      Raphaely           30.0       Purchasing
9        Mavris           40.0  Human Resources
10        Grant           50.0         Shipping
11     OConnell           50.0         Shipping
12       Feeney           50.0         Shipping
13        Walsh           50.0         Shipping
14        Jones           50.0         Shipping
15       McCain           50.0         Shipping
16      Everett           50.0         Shipping
17         Bell           50.0         Shipping
18      Perkins           50.0         Shipping
19        Gates           50.0         S

### < RIGHT OUTER JOIN >

In [14]:
df = pd.read_sql("""
        SELECT e.last_name, e.department_id, d.department_name
        FROM employees e,departments d
        WHERE e.department_id(+)= d.department_id
        """,xedb)
print(df)

      LAST_NAME  DEPARTMENT_ID       DEPARTMENT_NAME
0        Whalen           10.0        Administration
1           Fay           20.0             Marketing
2     Hartstein           20.0             Marketing
3        Tobias           30.0            Purchasing
4    Colmenares           30.0            Purchasing
5         Baida           30.0            Purchasing
6      Raphaely           30.0            Purchasing
7          Khoo           30.0            Purchasing
8        Himuro           30.0            Purchasing
9        Mavris           40.0       Human Resources
10       Feeney           50.0              Shipping
11       Fleaur           50.0              Shipping
12        Fripp           50.0              Shipping
13        Gates           50.0              Shipping
14          Gee           50.0              Shipping
15        Geoni           50.0              Shipping
16        Grant           50.0              Shipping
17        Jones           50.0              Sh

In [15]:
df = pd.read_sql("""
        SELECT e.last_name, e.department_id, d.department_name
        FROM employees e
        RIGHT OUTER JOIN departments d
        ON (e.department_id = d.department_id)
        """,xedb)
print(df)

      LAST_NAME  DEPARTMENT_ID       DEPARTMENT_NAME
0        Whalen           10.0        Administration
1           Fay           20.0             Marketing
2     Hartstein           20.0             Marketing
3        Tobias           30.0            Purchasing
4    Colmenares           30.0            Purchasing
5         Baida           30.0            Purchasing
6      Raphaely           30.0            Purchasing
7          Khoo           30.0            Purchasing
8        Himuro           30.0            Purchasing
9        Mavris           40.0       Human Resources
10       Feeney           50.0              Shipping
11       Fleaur           50.0              Shipping
12        Fripp           50.0              Shipping
13        Gates           50.0              Shipping
14          Gee           50.0              Shipping
15        Geoni           50.0              Shipping
16        Grant           50.0              Shipping
17        Jones           50.0              Sh

### < FULL OUTER JOIN >

In [16]:
df = pd.read_sql("""
        SELECT e.last_name, e.department_id, d.department_name
        FROM employees e, departments d
        WHERE e.department_id(+) = d.department_id(+)
        """,xedb)

ERROR:root:An unexpected error occurred while tokenizing input
The following traceback may be corrupted or invalid
The error message is: ('EOF in multi-line string', (1, 8))



DatabaseError: Execution failed on sql '
        SELECT e.last_name, e.department_id, d.department_name
        FROM employees e, departments d
        WHERE e.department_id(+) = d.department_id(+)
        ': ORA-01468: a predicate may reference only one outer-joined table

### FULL OUTER JOIN은  (+) 부호를 붙이면 오류가 난다
ORA-01468: a predicate may reference only one outer-joined table 

In [17]:
df = pd.read_sql("""
        SELECT e.last_name, e.department_id, d.department_name
        FROM employees e
        FULL OUTER JOIN departments d\
        ON (e.department_id = d.department_id)
        """,xedb)
print(df)

       LAST_NAME  DEPARTMENT_ID       DEPARTMENT_NAME
0           King           90.0             Executive
1        Kochhar           90.0             Executive
2        De Haan           90.0             Executive
3         Hunold           60.0                    IT
4          Ernst           60.0                    IT
5         Austin           60.0                    IT
6      Pataballa           60.0                    IT
7        Lorentz           60.0                    IT
8      Greenberg          100.0               Finance
9         Faviet          100.0               Finance
10          Chen          100.0               Finance
11       Sciarra          100.0               Finance
12         Urman          100.0               Finance
13          Popp          100.0               Finance
14      Raphaely           30.0            Purchasing
15          Khoo           30.0            Purchasing
16         Baida           30.0            Purchasing
17        Tobias           3

## 4-4 SELF JOIN
* 자신의 테이블을 참조해야 할 때 사용

#### 사원의 employee_id, last_name, 해당 관리자의 employee_id, last_name 조회

In [18]:
df = pd.read_sql("""
        SELECT employee_id, last_name, manager_id
        FROM employees
        """,xedb)
print(df)

     EMPLOYEE_ID    LAST_NAME  MANAGER_ID
0            100         King         NaN
1            101      Kochhar       100.0
2            102      De Haan       100.0
3            103       Hunold       102.0
4            104        Ernst       103.0
5            105       Austin       103.0
6            106    Pataballa       103.0
7            107      Lorentz       103.0
8            108    Greenberg       101.0
9            109       Faviet       108.0
10           110         Chen       108.0
11           111      Sciarra       108.0
12           112        Urman       108.0
13           113         Popp       108.0
14           114     Raphaely       100.0
15           115         Khoo       114.0
16           116        Baida       114.0
17           117       Tobias       114.0
18           118       Himuro       114.0
19           119   Colmenares       114.0
20           120        Weiss       100.0
21           121        Fripp       100.0
22           122     Kaufling     

In [23]:
# WHERE m.employee_id = w.manager_id

df = pd.read_sql("""
        SELECT w.employee_id, w.last_name, m.employee_id, m.last_name
        FROM employees w, employees m
        WHERE m.employee_id = w.manager_id
        ORDER BY w.employee_id
        """,xedb)
print(df)

     EMPLOYEE_ID    LAST_NAME  EMPLOYEE_ID  LAST_NAME
0            101      Kochhar          100       King
1            102      De Haan          100       King
2            103       Hunold          102    De Haan
3            104        Ernst          103     Hunold
4            105       Austin          103     Hunold
5            106    Pataballa          103     Hunold
6            107      Lorentz          103     Hunold
7            108    Greenberg          101    Kochhar
8            109       Faviet          108  Greenberg
9            110         Chen          108  Greenberg
10           111      Sciarra          108  Greenberg
11           112        Urman          108  Greenberg
12           113         Popp          108  Greenberg
13           114     Raphaely          100       King
14           115         Khoo          114   Raphaely
15           116        Baida          114   Raphaely
16           117       Tobias          114   Raphaely
17           118       Himur

In [26]:
# WHERE w.employee_id= m.manager_id

df = pd.read_sql("""
        SELECT w.employee_id, w.last_name, m.employee_id, m.last_name
        FROM employees w, employees m
        WHERE w.employee_id= m.manager_id
        ORDER BY w.employee_id
        """,xedb)
print(df)

     EMPLOYEE_ID  LAST_NAME  EMPLOYEE_ID   LAST_NAME
0            100       King          101     Kochhar
1            100       King          102     De Haan
2            100       King          114    Raphaely
3            100       King          120       Weiss
4            100       King          121       Fripp
5            100       King          122    Kaufling
6            100       King          123     Vollman
7            100       King          124     Mourgos
8            100       King          145     Russell
9            100       King          146    Partners
10           100       King          147   Errazuriz
11           100       King          148   Cambrault
12           100       King          149     Zlotkey
13           100       King          201   Hartstein
14           101    Kochhar          108   Greenberg
15           101    Kochhar          200      Whalen
16           101    Kochhar          203      Mavris
17           101    Kochhar          204      

In [25]:
df = pd.read_sql("""
                 SELECT w.employee_id, w.last_name, m.employee_id, m.last_name
                 FROM employees w, employees m
                 WHERE w.manager_id = m.employee_id(+)
                 ORDER BY w.employee_id
                 """,xedb)
print(df)

     EMPLOYEE_ID    LAST_NAME  EMPLOYEE_ID  LAST_NAME
0            100         King          NaN       None
1            101      Kochhar        100.0       King
2            102      De Haan        100.0       King
3            103       Hunold        102.0    De Haan
4            104        Ernst        103.0     Hunold
5            105       Austin        103.0     Hunold
6            106    Pataballa        103.0     Hunold
7            107      Lorentz        103.0     Hunold
8            108    Greenberg        101.0    Kochhar
9            109       Faviet        108.0  Greenberg
10           110         Chen        108.0  Greenberg
11           111      Sciarra        108.0  Greenberg
12           112        Urman        108.0  Greenberg
13           113         Popp        108.0  Greenberg
14           114     Raphaely        100.0       King
15           115         Khoo        114.0   Raphaely
16           116        Baida        114.0   Raphaely
17           117       Tobia

## 4-5 JOIN USING

In [27]:
df = pd.read_sql("""
        SELECT l.city, d.department_name
        FROM locations l
        JOIN departments d 
        USING (location_id)
        WHERE location_id = 1400
        """,xedb)
print(df)

        CITY DEPARTMENT_NAME
0  Southlake              IT


In [28]:
df = pd.read_sql("""
        SELECT city, department_name
        FROM locations 
        JOIN departments 
        USING (location_id)
        WHERE location_id = 1400
        """,xedb)
print(df)

        CITY DEPARTMENT_NAME
0  Southlake              IT


***
# 더 연습문제 
***

#### (1) 모든 사원의 last_name, department_id, department_name을 조회하세요.

In [29]:
df = pd.read_sql("""
        SELECT e.last_name, e.department_id, d.department_name
        FROM employees e, departments d
        WHERE e.department_id = d.department_id
        """,xedb)
print(df)

#부서배치가 안 된 사원은 누락되어 조회된다

      LAST_NAME  DEPARTMENT_ID  DEPARTMENT_NAME
0        Whalen             10   Administration
1           Fay             20        Marketing
2     Hartstein             20        Marketing
3        Tobias             30       Purchasing
4    Colmenares             30       Purchasing
5         Baida             30       Purchasing
6      Raphaely             30       Purchasing
7          Khoo             30       Purchasing
8        Himuro             30       Purchasing
9        Mavris             40  Human Resources
10       Feeney             50         Shipping
11       Fleaur             50         Shipping
12        Fripp             50         Shipping
13        Gates             50         Shipping
14          Gee             50         Shipping
15        Geoni             50         Shipping
16        Grant             50         Shipping
17        Jones             50         Shipping
18     Kaufling             50         Shipping
19       Ladwig             50         S

#### (2) 부서배치가 안 된 사원을 포함하여 모든 사원의 last_name, department_id, department_name을 조회하세요.

In [30]:
df = pd.read_sql("""
        SELECT e.last_name, e.department_id, d.department_name
        FROM employees e, departments d
        WHERE e.department_id = d.department_id(+)
        """,xedb)
print(df)

      LAST_NAME  DEPARTMENT_ID  DEPARTMENT_NAME
0        Whalen           10.0   Administration
1           Fay           20.0        Marketing
2     Hartstein           20.0        Marketing
3    Colmenares           30.0       Purchasing
4        Himuro           30.0       Purchasing
5        Tobias           30.0       Purchasing
6         Baida           30.0       Purchasing
7          Khoo           30.0       Purchasing
8      Raphaely           30.0       Purchasing
9        Mavris           40.0  Human Resources
10        Grant           50.0         Shipping
11     OConnell           50.0         Shipping
12       Feeney           50.0         Shipping
13        Walsh           50.0         Shipping
14        Jones           50.0         Shipping
15       McCain           50.0         Shipping
16      Everett           50.0         Shipping
17         Bell           50.0         Shipping
18      Perkins           50.0         Shipping
19        Gates           50.0         S

#### (3) 부서 80에 속하는 사원의 last_name, job_id, department_name, location_id을 조회하세요.

In [31]:
df = pd.read_sql("""
        SELECT  e.last_name, e.job_id, d.department_name,d.location_id
        FROM employees e, departments d
        WHERE e.department_id = d.department_id
        AND d.department_id = 80
        """,xedb)
print(df)

     LAST_NAME  JOB_ID DEPARTMENT_NAME  LOCATION_ID
0      Russell  SA_MAN           Sales         2500
1     Partners  SA_MAN           Sales         2500
2    Errazuriz  SA_MAN           Sales         2500
3    Cambrault  SA_MAN           Sales         2500
4      Zlotkey  SA_MAN           Sales         2500
5       Tucker  SA_REP           Sales         2500
6    Bernstein  SA_REP           Sales         2500
7         Hall  SA_REP           Sales         2500
8        Olsen  SA_REP           Sales         2500
9    Cambrault  SA_REP           Sales         2500
10     Tuvault  SA_REP           Sales         2500
11        King  SA_REP           Sales         2500
12       Sully  SA_REP           Sales         2500
13      McEwen  SA_REP           Sales         2500
14       Smith  SA_REP           Sales         2500
15       Doran  SA_REP           Sales         2500
16      Sewall  SA_REP           Sales         2500
17     Vishney  SA_REP           Sales         2500
18      Gree

In [32]:
df = pd.read_sql("""
        SELECT  e.last_name,e.job_id, d.department_name,d.location_id
        FROM employees e, departments d
        WHERE e.department_id = 80
        AND d.department_id = 80
        """,xedb)
print(df)

     LAST_NAME  JOB_ID DEPARTMENT_NAME  LOCATION_ID
0      Russell  SA_MAN           Sales         2500
1     Partners  SA_MAN           Sales         2500
2    Errazuriz  SA_MAN           Sales         2500
3    Cambrault  SA_MAN           Sales         2500
4      Zlotkey  SA_MAN           Sales         2500
5       Tucker  SA_REP           Sales         2500
6    Bernstein  SA_REP           Sales         2500
7         Hall  SA_REP           Sales         2500
8        Olsen  SA_REP           Sales         2500
9    Cambrault  SA_REP           Sales         2500
10     Tuvault  SA_REP           Sales         2500
11        King  SA_REP           Sales         2500
12       Sully  SA_REP           Sales         2500
13      McEwen  SA_REP           Sales         2500
14       Smith  SA_REP           Sales         2500
15       Doran  SA_REP           Sales         2500
16      Sewall  SA_REP           Sales         2500
17     Vishney  SA_REP           Sales         2500
18      Gree

#### (4) commission을 지급받는 사원들의last_name, department_name, location_id, city를 조회하세요

In [33]:
df = pd.read_sql("""
        SELECT e.last_name, d.department_name, d.location_id, l.city
        FROM employees e, departments d, locations l
        WHERE e.department_id = d.department_id(+)
        AND  d.location_id = l.location_id(+)
        AND e.commission_pct IS NOT NULL
        """,xedb)
print(df)

     LAST_NAME DEPARTMENT_NAME  LOCATION_ID    CITY
0      Russell           Sales       2500.0  Oxford
1     Partners           Sales       2500.0  Oxford
2    Errazuriz           Sales       2500.0  Oxford
3    Cambrault           Sales       2500.0  Oxford
4      Zlotkey           Sales       2500.0  Oxford
5       Tucker           Sales       2500.0  Oxford
6    Bernstein           Sales       2500.0  Oxford
7         Hall           Sales       2500.0  Oxford
8        Olsen           Sales       2500.0  Oxford
9    Cambrault           Sales       2500.0  Oxford
10     Tuvault           Sales       2500.0  Oxford
11        King           Sales       2500.0  Oxford
12       Sully           Sales       2500.0  Oxford
13      McEwen           Sales       2500.0  Oxford
14       Smith           Sales       2500.0  Oxford
15       Doran           Sales       2500.0  Oxford
16      Sewall           Sales       2500.0  Oxford
17     Vishney           Sales       2500.0  Oxford
18      Gree

#### (5) last_name에 'a'가 포함된 모든 사원의 last_name, department_name을 조회하세요.

In [34]:
df = pd.read_sql("""
        SELECT e.last_name, d.department_name
        FROM employees e, departments d
        WHERE e.department_id = d.department_id(+)
        AND instr(e.last_name,'a') > 0
        """,xedb)
print(df)

     LAST_NAME   DEPARTMENT_NAME
0       Whalen    Administration
1          Fay         Marketing
2    Hartstein         Marketing
3   Colmenares        Purchasing
4       Tobias        Purchasing
5        Baida        Purchasing
6     Raphaely        Purchasing
7       Mavris   Human Resources
8        Grant          Shipping
9        Walsh          Shipping
10      McCain          Shipping
11       Gates          Shipping
12      Cabrio          Shipping
13    Sarchand          Shipping
14    Sullivan          Shipping
15      Fleaur          Shipping
16      Taylor          Shipping
17      Vargas          Shipping
18       Matos          Shipping
19      Davies          Shipping
20        Rajs          Shipping
21       Patel          Shipping
22      Ladwig          Shipping
23  Philtanker          Shipping
24      Mallin          Shipping
25      Marlow          Shipping
26      Markle          Shipping
27      Landry          Shipping
28       Nayer          Shipping
29     Vol

In [35]:
df = pd.read_sql("""
        SELECT e.last_name, d.department_name
        FROM employees e 
        LEFT OUTER JOIN departments d
        ON e.department_id = d.department_id
        WHERE e.last_name LIKE '%a%'
        """,xedb)
print(df)

     LAST_NAME   DEPARTMENT_NAME
0       Whalen    Administration
1          Fay         Marketing
2    Hartstein         Marketing
3   Colmenares        Purchasing
4       Tobias        Purchasing
5        Baida        Purchasing
6     Raphaely        Purchasing
7       Mavris   Human Resources
8        Grant          Shipping
9        Walsh          Shipping
10      McCain          Shipping
11       Gates          Shipping
12      Cabrio          Shipping
13    Sarchand          Shipping
14    Sullivan          Shipping
15      Fleaur          Shipping
16      Taylor          Shipping
17      Vargas          Shipping
18       Matos          Shipping
19      Davies          Shipping
20        Rajs          Shipping
21       Patel          Shipping
22      Ladwig          Shipping
23  Philtanker          Shipping
24      Mallin          Shipping
25      Marlow          Shipping
26      Markle          Shipping
27      Landry          Shipping
28       Nayer          Shipping
29     Vol

#### (6) toronto도시(location테이블, city컬럼)에서 근무하는 모든 사원의 last_name, job_id, department_id, department_name을 조회하세요.

In [37]:
df = pd.read_sql("""
        SELECT e.last_name, e.job_id, e.department_id,d.department_name
        FROM employees e, departments d, locations l
        WHERE e.department_id = d.department_id
        AND d.location_id = l.location_id
        AND l.city = 'Toronto'
        """,xedb)
print(df)

   LAST_NAME  JOB_ID  DEPARTMENT_ID DEPARTMENT_NAME
0  Hartstein  MK_MAN             20       Marketing
1        Fay  MK_REP             20       Marketing


#### (7) 2006년도에 입사한 사원들의 부서이름별 급여총액과 평균급여를 출력하세요.

In [38]:
df = pd.read_sql("""
        SELECT d.department_name, sum(e.salary) sumsal, round(avg(e.salary),0) avgsal
        FROM employees e, departments d
        WHERE e.department_id = d.department_id
        AND e.hire_date >= to_date('2006-01-01','YYYY-MM-DD')
        AND e.hire_date < to_date('2007-01-01','YYYY-MM-DD')
        GROUP BY d.department_name
        """,xedb)
print(df)

  DEPARTMENT_NAME  SUMSAL  AVGSAL
0      Purchasing    2600    2600
1              IT   13800    6900
2        Shipping   37800    2908
3           Sales   59100    8443
4         Finance    7800    7800


#### (8) 2006년도에 입사한 사원들의 도시이름별 급여총액과 평균급여를 출력하세요.

In [39]:
df = pd.read_sql("""
        SELECT l.city, sum(e.salary) sumsal, round(avg(e.salary),0) avgsal
        FROM employees e, departments d, locations l
        WHERE e.department_id = d.department_id
        AND d.location_id = l.location_id
        AND e.hire_date >= to_date('2006-01-01','YYYY-MM-DD')
        AND e.hire_date < to_date('2007-01-01','YYYY-MM-DD')
        GROUP BY l.city
        """,xedb)
print(df)

                  CITY  SUMSAL  AVGSAL
0              Seattle   10400    5200
1  South San Francisco   37800    2908
2            Southlake   13800    6900
3               Oxford   59100    8443


#### (9) 2007년도에 입사한 사원들의 도시이름별로 급여총액과 평균급여를 출력하세요.(부서 배치를 받지 않는 사원을 포함)

In [40]:
df = pd.read_sql("""
        SELECT l.city, sum(e.salary) sumsal, round(avg(e.salary),0) avgsal
        FROM employees e, departments d, locations l
        WHERE e.department_id = d.department_id(+)
        AND d.location_id = l.location_id(+)
        AND e.hire_date >= to_date('2007-01-01','YYYY-MM-DD')
        AND e.hire_date < to_date('2008-01-01','YYYY-MM-DD')
        GROUP BY l.city
        """,xedb)
print(df)

                  CITY  SUMSAL  AVGSAL
0                 None    7000    7000
1              Seattle    9400    4700
2  South San Francisco   26100    2900
3            Southlake   10200    5100
4               Oxford   42200    8440


In [ ]:
cur.close()
xedb.close()